# Installation of packages

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python pygame

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install azure-storage-blob

Note: you may need to restart the kernel to use updated packages.


# Main Code

In [ ]:
import os
import cv2
import pygame
import numpy as np
import time
import datetime

def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def adjust_brightness(image, brightness):
    return cv2.convertScaleAbs(image, alpha=1.0, beta=brightness)

def check_person_status(person_id, person_bbox, current_time):
    x, y, w, h = person_bbox
    person_center = (int(x + w / 2), int(y + h / 2))
    
    left_of_grey_area_x_start = boundary_zone[0]
    left_of_grey_area_x_end = buffer_zone[0]
    
    in_left_of_grey_area = (left_of_grey_area_x_start < person_center[0] < left_of_grey_area_x_end)
    
    if person_id not in persons:
        persons[person_id] = {
            'start_time': current_time,
            'in_left_of_grey_area': in_left_of_grey_area,
            'loitering': False,  
            'out_of_boundary': False,  
            'last_time_out_of_boundary': None  
        }
    else:
        person_record = persons[person_id]
        if in_left_of_grey_area:
            if not person_record['in_left_of_grey_area']:
                person_record['start_time'] = current_time
            elif current_time - person_record['start_time'] > 5:
                pygame.mixer.Sound.play(alert_sound)
            person_record['in_left_of_grey_area'] = True
        else:
            person_record['in_left_of_grey_area'] = False
    
    boundary_polygon = np.array([
        [boundary_zone[0], boundary_zone[1]],
        [boundary_zone[0] + boundary_zone[2], boundary_zone[1]],
        [boundary_zone[0] + boundary_zone[2], boundary_zone[1] + boundary_zone[3]],
        [boundary_zone[0], boundary_zone[1] + boundary_zone[3]]
    ], dtype=np.int32)

    buffer_polygon = np.array([
        [buffer_zone[0], buffer_zone[1]],
        [buffer_zone[0] + buffer_zone[2], buffer_zone[1]],
        [buffer_zone[0] + buffer_zone[2], buffer_zone[1] + buffer_zone[3]],
        [buffer_zone[0], buffer_zone[1] + buffer_zone[3]]
    ], dtype=np.int32)
    
    inside_boundary = cv2.pointPolygonTest(boundary_polygon, person_center, False) >= 0
    inside_buffer = cv2.pointPolygonTest(buffer_polygon, person_center, False) >= 0

    if person_id not in persons:
        persons[person_id] = {
            'start_time': current_time, 
            'loitering': False, 
            'out_of_boundary': not inside_boundary,
            'last_time_out_of_boundary': None 
        }
    else:
        if not inside_boundary:
            persons[person_id]['out_of_boundary'] = True
            persons[person_id]['last_time_out_of_boundary'] = current_time  
        else:
            persons[person_id]['out_of_boundary'] = False
            persons[person_id]['last_time_out_of_boundary'] = None  
            
    in_left_of_grey_area = person_center[0] > boundary_zone_x and person_center[0] < buffer_zone_x
    return in_left_of_grey_area

def is_night_time(start_hour, start_minute, end_hour, end_minute):
    current_datetime = datetime.datetime.now()
    start_time = datetime.datetime(current_datetime.year, current_datetime.month, current_datetime.day, start_hour, start_minute)
    end_time = datetime.datetime(current_datetime.year, current_datetime.month, current_datetime.day, end_hour, end_minute)

    return start_time <= current_datetime <= end_time or (end_time < start_time and not (end_time < current_datetime < start_time))

def get_night_mode_time_range():
    while True:
        try:
            print("SET TIME RANGE FOR NIGHT VISION MODE:")
            start_hour = int(input("Enter start hour (0-23): "))
            if not 0 <= start_hour <= 23:
                raise ValueError("Hour must be between 0 and 23.")
            
            start_minute = int(input("Enter start minute (0-59): "))
            if not 0 <= start_minute <= 59:
                raise ValueError("Minute must be between 0 and 59.")
            
            end_hour = int(input("Enter end hour (0-23): "))
            if not 0 <= end_hour <= 23:
                raise ValueError("Hour must be between 0 and 23.")
            
            end_minute = int(input("Enter end minute (0-59): "))
            if not 0 <= end_minute <= 59:
                raise ValueError("Minute must be between 0 and 59.")
            
            return start_hour, start_minute, end_hour, end_minute
        except ValueError as e:
            print(f"Invalid input: {e}. Please enter valid values.")
            
print("DIFFERENT MODES FOR BODY DETECTION")
print("Type '1' for Full Body Detection")
print("Type '2' for Upper Body Detection")
print("Type '3' for Frontal Face Detection")
mode = int(input("Enter Mode: "))



if mode == 1:
    face_cascade_path = "./haarcascade_fullbody.xml"
elif mode == 2:
    face_cascade_path = "./haarcascade_upperbody.xml"
elif mode == 3:
    face_cascade_path = "./haarcascade_frontalface_default.xml"
else:
    print("Choose a correct mode")
    exit()


# Initialize Pygame for alert
pygame.init()
alert_sound = pygame.mixer.Sound('buzzer.wav')



#-------------------------------------------------------------------------------------------------------------------------------
# INITIALISE THE WEBCAM HERE

cap = cv2.VideoCapture(0) # webcam index is either 1 or 0, pls try both if unsure
print("Webcam opened:", cap.isOpened())

#-------------------------------------------------------------------------------------------------------------------------------


#--------------------------------------------------------------------------------------------------------------------------------    
# Check and create 'Video footages' folder, if no such folder exists it will be created automatically

video_folder = "Video footages"  # default folder is Video footages, but feel free to rename
if not os.path.exists(video_folder):
    os.makedirs(video_folder)

#--------------------------------------------------------------------------------------------------------------------------------    
    
    

# Define filename for video saving
current_datetime = datetime.datetime.now()
formatted_date = current_datetime.strftime("%Y%m%d_%H%M%S")
video_filename = f"{video_folder}/recording_{formatted_date}.avi"

# Define the codec and create VideoWriter object
# Adjust the resolution (640, 480) to match your webcam's resolution
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(video_filename, fourcc, 20.0, (640, 480))
    

#face_cascade_path = './haarcascade_upperbody-Copy1.xml'
face_cascade = cv2.CascadeClassifier(face_cascade_path)

# Get night mode time range from user
night_range_start_hour, night_range_start_minute, night_range_end_hour, night_range_end_minute = get_night_mode_time_range()

boundary_zone = [150, 100, 300, 220]
buffer_zone = [boundary_zone[0] + 10, boundary_zone[1] + 10, boundary_zone[2] - 20, boundary_zone[3] - 20]
loitering_threshold = 5  # 5 sec

persons = {}


while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    # Flag to indicate whether the code block should be executed
    execute_code = is_night_time(night_range_start_hour, night_range_start_minute, night_range_end_hour, night_range_end_minute)    
        
    if execute_code:
        current_time = time.time()
        
        current_datetime = datetime.datetime.now()
        formatted_datetime = current_datetime.strftime("DATE: %d/%m/%Y %H:%M:%S")


        cv2.putText(frame, formatted_datetime, (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)        
        
        height, width = frame.shape[:2]

        expansion_width = 70

        boundary_zone_width = (width // 2) + expansion_width
        boundary_zone_height = height
        boundary_zone_x = (width // 2) - expansion_width
        boundary_zone_y = 0

        buffer_zone_width = boundary_zone_width // 2
        buffer_zone_height = boundary_zone_height
        buffer_zone_x = boundary_zone_x + buffer_zone_width - expansion_width // 2
        buffer_zone_y = boundary_zone_y

        boundary_zone = [boundary_zone_x, boundary_zone_y, boundary_zone_width, boundary_zone_height]
        buffer_zone = [buffer_zone_x, buffer_zone_y, buffer_zone_width, buffer_zone_height]

        boundary_polygon = np.array([
            [boundary_zone[0], boundary_zone[1]],
            [boundary_zone[0] + boundary_zone[2], boundary_zone[1]],
            [boundary_zone[0] + boundary_zone[2], boundary_zone[1] + boundary_zone[3]],
            [boundary_zone[0], boundary_zone[1] + boundary_zone[3]]
        ], np.int32)

        buffer_polygon = np.array([
            [buffer_zone[0], buffer_zone[1]],
            [buffer_zone[0] + buffer_zone[2], buffer_zone[1]],
            [buffer_zone[0] + buffer_zone[2], buffer_zone[1] + buffer_zone[3]],
            [buffer_zone[0], buffer_zone[1] + buffer_zone[3]]
        ], np.int32)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(image=gray, scaleFactor=1.05, minNeighbors=6, minSize=(20, 20))
        current_time = time.time()

        any_out_of_boundary = False
        boundary_color = (0, 255, 0)

        for (x, y, w, h) in faces:
            person_id = (x, y, w, h)
            check_person_status(person_id, (x, y, w, h), current_time)

            person = persons[person_id]

            if person['out_of_boundary']:
                box_color = (0, 0, 255)
                boundary_color = (0, 0, 255)
                if person['last_time_out_of_boundary'] and (current_time - person['last_time_out_of_boundary']) < 1:
                    pygame.mixer.Sound.play(alert_sound)
            elif person['loitering']:
                box_color = (0, 165, 255)
                if current_time - person['start_time'] >= loitering_threshold:
                    pygame.mixer.Sound.play(alert_sound)
            else:
                box_color = (255, 0, 0)
                person['loitering'] = False  # Reset loitering status

        for (x, y, w, h) in faces:
            person_id = (x, y, w, h)
            in_left_of_grey_area = check_person_status(person_id, (x, y, w, h), current_time)

            # Determine the color of the detection box based on the person's location
            if in_left_of_grey_area:
                box_color = (0, 165, 255)  # Orange color in BGR
            elif person['out_of_boundary']:
                box_color = (0, 0, 255)  # Red color in BGR
            else:
                box_color = (255, 0, 0)  # Blue color in BGR
        
        
            cv2.rectangle(frame, (x, y), (x + w, y + h), box_color, 3)
            
            
        cv2.polylines(frame, [boundary_polygon], True, (0, 255, 0), 2)
        
        cv2.polylines(frame, [buffer_polygon], True, (128, 128, 128), 2)

        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gamma_corrected_frame = adjust_gamma(gray_frame, gamma=1.0)
        brightened_frame = adjust_brightness(gamma_corrected_frame, brightness=30)
        
        
        
        cv2.imshow('Live Webcam Feed', brightened_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    else:
            current_datetime = datetime.datetime.now()
            formatted_datetime = current_datetime.strftime("DATE: %d/%m/%Y %H:%M:%S")
            cv2.putText(frame, formatted_datetime, (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            height, width = frame.shape[:2]
            
            expansion_width = 70

            boundary_zone_width = (width // 2) + expansion_width
            boundary_zone_height = height
            boundary_zone_x = (width // 2) - expansion_width
            boundary_zone_y = 0

            buffer_zone_width = boundary_zone_width // 2
            buffer_zone_height = boundary_zone_height
            buffer_zone_x = boundary_zone_x + buffer_zone_width - expansion_width // 2
            buffer_zone_y = boundary_zone_y

            boundary_zone = [boundary_zone_x, boundary_zone_y, boundary_zone_width, boundary_zone_height]
            buffer_zone = [buffer_zone_x, buffer_zone_y, buffer_zone_width, buffer_zone_height]

            boundary_polygon = np.array([
                [boundary_zone[0], boundary_zone[1]],
                [boundary_zone[0] + boundary_zone[2], boundary_zone[1]],
                [boundary_zone[0] + boundary_zone[2], boundary_zone[1] + boundary_zone[3]],
                [boundary_zone[0], boundary_zone[1] + boundary_zone[3]]
            ], np.int32)

            buffer_polygon = np.array([
                [buffer_zone[0], buffer_zone[1]],
                [buffer_zone[0] + buffer_zone[2], buffer_zone[1]],
                [buffer_zone[0] + buffer_zone[2], buffer_zone[1] + buffer_zone[3]],
                [buffer_zone[0], buffer_zone[1] + buffer_zone[3]]
            ], np.int32)

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(image=gray, scaleFactor=1.1, minNeighbors=6, minSize=(30, 30))
            current_time = time.time()

            any_out_of_boundary = False
            boundary_color = (0, 255, 0)

            for (x, y, w, h) in faces:
                person_id = (x, y, w, h)
                check_person_status(person_id, (x, y, w, h), current_time)

                person = persons[person_id]

                if person['out_of_boundary']:
                    box_color = (0, 0, 255)
                    boundary_color = (0, 0, 255)
                    if person['last_time_out_of_boundary'] and (current_time - person['last_time_out_of_boundary']) < 1:
                        pygame.mixer.Sound.play(alert_sound)
                elif person['loitering']:
                    box_color = (0, 165, 255)
                    if current_time - person['start_time'] >= loitering_threshold:
                        pygame.mixer.Sound.play(alert_sound)
                else:
                    box_color = (255, 0, 0)
                    person['loitering'] = False  # Reset loitering status

                    
            for (x, y, w, h) in faces:
                person_id = (x, y, w, h)
                in_left_of_grey_area = check_person_status(person_id, (x, y, w, h), current_time)

                # Determine the color of the detection box based on the person's location
                if in_left_of_grey_area:
                    box_color = (0, 165, 255)  # Orange color in BGR
                elif person['out_of_boundary']:
                    box_color = (0, 0, 255)  # Red color in BGR
                else:
                    box_color = (255, 0, 0)  # Blue color in BGR
                    
                cv2.rectangle(frame, (x, y), (x + w, y + h), box_color, 3)

            cv2.polylines(frame, [boundary_polygon], True, (0, 255, 0), 2)
            cv2.polylines(frame, [buffer_polygon], True, (128, 128, 128), 2)
            

            cv2.imshow('Live Webcam Feed', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    out.write(frame)



cap.release()
out.release()
cv2.destroyAllWindows()
pygame.quit()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
DIFFERENT MODES FOR BODY DETECTION
Type '1' for Full Body Detection
Type '2' for Upper Body Detection
Type '3' for Frontal Face Detection
Enter Mode: 1
Webcam opened: True
SET TIME RANGE FOR NIGHT VISION MODE:
Enter start hour (0-23): 20
Enter start minute (0-59): 0
Enter end hour (0-23): 7


# Azure Blob Storage Service: Send selective cctv footages to Azure Blob Explorer

In [ ]:
from azure.storage.blob import BlobServiceClient
import os

def get_azure_storage_config():
    print("Enter Azure Storage Configuration:")
    connection_string = input("Enter connection string: ")
    container_name = input("Enter container name: ")
    video_file_name = input("Enter video file path (eg [./videos/12345678.avi] remove the '.' at the front if this notebook is not in the same directory as the folder 'videos':")

    return connection_string, container_name, video_file_name

# Get Azure Storage configuration from the user
connection_string, container_name, video_file_name = get_azure_storage_config()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to a container. Does not make a network call.
container_client = blob_service_client.get_container_client(container_name)

# OPTIONAL: Create a new container if it does not exist
# Note: This makes a network call.
try:
    container_client.create_container()
except Exception as e:
    print(f"Container might already exist or another error occurred: {e}")

blob_name = os.path.basename(video_file_name)  
blob_client = container_client.get_blob_client(blob_name)

# Upload the file
with open(video_file_name, "rb") as data:
    blob_client.upload_blob(data)